In [1]:
from google.colab import files
uploaded = files.upload()

Saving vulcancoin-part2.csv to vulcancoin-part2.csv
Saving vulcancoin-part1.csv to vulcancoin-part1.csv


In [2]:
pip install datetime

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 256kB 8.0MB/s 


In [3]:
import datetime
import pandas as pd
import re
import numpy as np

In [4]:
df = pd.read_csv('vulcancoin-part2.csv')

In [5]:
df2 = pd.read_csv('vulcancoin-part1.csv')

In [6]:
df3 = pd.concat([df, df2])

How many total transactions are in the data?


In [7]:
df3.shape

(1109837, 5)

1,109,837 total transactions. 


## The 400,000 transaction in the dataset takes place during which month?

In [8]:
df3['date'] = pd.to_datetime(df3['date'])

In [9]:
df3 = df3.sort_values('date')

In [10]:
df3.reset_index(drop=True, inplace=True)

In [11]:
df3.iloc[400000]

date         2019-04-08 00:00:00
tx_from                       E3
tx_to                         E6
tx_amount                 82 VLC
tx_fee                         4
Name: 400000, dtype: object

April.

## What is the average daily transaction volume during December 2018?

In [12]:
filtered_df = df3.loc[(df3['date'].dt.month == 12) & (df3['date'].dt.year == 2018)]

In [13]:
filtered_df['day'] = filtered_df['date'].dt.day

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
filtered_df.groupby(['day']).count().mean()

date         2123.612903
tx_from      2123.612903
tx_to        2123.612903
tx_amount    2123.612903
tx_fee       2082.354839
dtype: float64

Average daily volume: 2123

## What is the R-squared value between transaction amount and transaction fee? Include supply transactions from the Central Bank, where the fee isn't listed use 3 VLC.

In [15]:
def one_func(astr):
  numeric_string = re.sub("[^0-9]", "", astr)
  return numeric_string

In [16]:
df3['tx_amount'] = df3['tx_amount'].apply(one_func)

In [17]:
df3['tx_amount'] = df3['tx_amount'].astype(int)

In [18]:
df3['tx_fee'] = df3['tx_fee'].fillna(3)

In [19]:
df3['tx_amount'].corr(df3['tx_fee'])

0.8699586628194713

The r^2

## To find the market cap of Vulcan coin, we can multiply one VLC token by the total amount supplied by the central bank. If the market cap is $68,284,224.90, what is the value of 1 VLC?

In [20]:
df3_cb = df3['tx_from'] == 'Central Bank'

In [21]:
total_VLC = df3[df3_cb]['tx_amount'].sum()
market_cap = 68284224.90

In [22]:
value_VLC_token = (market_cap / total_VLC)

In [23]:
print(value_VLC_token)

6.15


## At the end of the provided data, which user has the lowest account balance? Remember to account for sender transaction fees. Assume that all users start with 0 balance. 


In [24]:
df3['sender_amount'] = df3['tx_amount'] + df3['tx_fee']

In [25]:
no_cb = df3[df3['tx_from'] != 'Central Bank']

In [26]:
grouped_df = no_cb.groupby(['tx_from'])

In [27]:
deductions = grouped_df['sender_amount'].agg(np.sum)

In [28]:
addition_grouped_df = df3.groupby(['tx_to'])

In [29]:
additions = addition_grouped_df['tx_amount'].agg(np.sum)

In [30]:
balance = additions - deductions

In [31]:
balance.sort_values()

tx_to
F7      17.0
B8      32.0
C3      39.0
G8      41.0
F6      41.0
       ...  
G5     793.0
G2     874.0
C8     877.0
D6     898.0
D5    1046.0
Length: 85, dtype: float64

The account with the lowest balance is F7